# 모듈 임포트

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings(action='ignore')
import pickle

# 설명 : make_season(감염병)

2010년부터 2019년까지 모든 감염병에 대한 데이터를 불러들여서  
봄 여름,가을,겨울 로 구분해 빈도수를 합하여
발생빈도 비율을 통해 계절별 위험 요인을 구함  


In [35]:
def make_season():

    # 빈도 가져워
    df = pd.read_csv('지역별_2010부터_감염병발생빈도.csv')

    # 월 추가
    df['월'] = df['날짜'].apply(lambda x : x[-2:])
    df['계절'] = df['월']
    for i in tqdm(range(len(df))):   # 32640 : 계절별 요인, 2020년은 고려하지 않았고 2010년부터 2019\년까지 
        if df['월'][i] in ['03','04','05']:
            df['계절'][i] = '봄'
        elif df['월'][i] in ['06','07','08']:
            df['계절'][i] = '여름'
        elif df['월'][i] in ['09','10','11']:
            df['계절'][i] = '가을'
        else:
            df['계절'][i] = '겨울'

    season = df.iloc[range(32640)].groupby('계절').sum().reset_index()
    with open('season.pickle', 'wb') as f:
        pickle.dump(season, f, pickle.HIGHEST_PROTOCOL) # save as season.pickle
    return season


# 필요한 데이터들

In [36]:
df = pd.read_csv('지역별_2010부터_감염병발생빈도.csv')
age = pd.read_csv('연령_전체.csv', encoding = 'euc-kr',header = None)
data = pd.read_csv('지역별_2018부터_10만명당감염병발생비율.csv')
prevention = pd.read_excel('감염병 예방.xlsx')
with open('season.pickle', 'rb') as f:
    season = pickle.load(f)

# 설명 : make_season_dict(감염병)

make_season으로 만들어진 데이터를 통해서  
특정 감염병에 대해  
발생빈도 비율을 통해 계절별 위험 요인을 구함  


In [372]:
def make_season_dict(disease= '수두'):
    season_dict = {}
    for i in range(len(season['계절'])):
        a =season['계절'][i]
        #season_dict[a] =( season[disease][i] / np.log(1+season[disease].sum())
        
        season_dict[a] =(season[disease][i] * np.log(len(season['계절'])))/(1+season[disease].sum())
    
    return season_dict

## tfidf 넣어서 계절별 요인 수 구하기

In [381]:
# tfidf 수정`
def make_season_dict2(disease= '수두'):
    season_dict = {}
    cnt = 0
    for i in range(len(season['계절'])):
        if season[disease][i] != 0:
            cnt += 1
    for i in range(len(season['계절'])):
        a =season['계절'][i]
        #season_dict[a] = (season[disease][i]) * (np.log(len(season['계절'])-1) / (1+cnt))
        season_dict[a] = season[disease][i] * (len(season['계절'])/(1+cnt))
       # (age[disease][i]*100000) * (np.log(len(age['나이'])))/(1+((age[disease]*100000).sum()))

       # season_dict[a] = np.log(1+season[disease][i]) * (np.log(len(season['계절'])/(1+cnt))
    return season_dict

In [382]:
make_season_dict2('쯔쯔가무시증')

{'가을': 70556.0, '겨울': 4852.0, '봄': 1798.0, '여름': 2534.0}

In [220]:
make_season_dict('쯔쯔가무시증')

{'가을': 3.0953668949720177,
 '겨울': 2.353193740563621,
 '봄': 2.078051430559366,
 '여름': 2.173140877006322}

# 설명 : make_age_dict(감염병)

2010년부터 2020년 8월까지 특정 감염병에 대한 데이터를 불러들여서  
나이대로 구분해 빈도수를 합하여
발생빈도 비율을 통해 나이별 위험 요인을 구함  


In [373]:
def make_age_dict(disease = '수두'):

    # 불러들이기
    age = pd.read_csv('연령_전체.csv', encoding = 'euc-kr',header = None)

    # 컬럼명 지정
    colum = ['나이','에볼라바이러스병 ','마버그열','라싸열','크리미안콩고출혈열','남아메리카출혈열','리프트밸리열','두창','페스트','탄저','보툴리눔독소증','야토병','신종감염병증후군',
            '중증급성호흡기증후군(SARS)','중동호흡기증후군(MERS)','동물인플루엔자 인체감염증','신종인플루엔자','디프테리아','수두','홍역','콜레라','장티푸스', '파라티푸스',
            '세균성이질','장출혈성대장균감염증','A형간염','백일해','유행성이하선염','풍진(2018년이전)','풍진(선천성)','풍진(후천성)','폴리오','수막구균 감염증','b형헤모필루스인플루엔자',
            '폐렴구균 감염증','한센병','성홍열','반코마이신내성황색포도알균(VRSA) 감염증','카바페넴내성장내세균속균종(CRE) 감염증','E형간염','파상풍','B형간염','일본뇌염',
            'C형간염','말라리아','레지오넬라증','비브리오패혈증','발진티푸스','발진열','쯔쯔가무시증','렙토스피라증','브루셀라증','공수병','신증후군출혈열','크로이츠펠트-야콥병(CJD) 및 변종크로이츠펠트-야콥병(vCJD)',
            '황열','뎅기열','큐열','웨스트나일열','라임병','진드기매개뇌염','유비저','치쿤구니야열','중증열성혈소판감소증후군(SFTS)','지카바이러스감염증']

    # age의 컬럼을 colum으로
    age.columns = colum


    ## 나이 요인 뽑기
    age_dict = {}
    for i in range(1, len(age['나이'])):
        a =age['나이'][i]
        #age_dict[a] =( age[disease][i] / np.log(1+age[disease][1:].sum()))
        age_dict[a] = (age[disease][i]*100000) * (np.log(len(age['나이'])-1))/(1+((age[disease]*100000).sum()))

    return age_dict

In [155]:
age = pd.read_csv('연령_전체.csv', encoding = 'euc-kr',header = None)
age

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,계,0,0,0,0,0,0,0,0,0,...,0,3.94,1.27,0.00,0.30,0,0.05,0.08,2.45,0.07
1,0~9세,0,0,0,0,0,0,0,0,0,...,0,0.54,0.02,0.00,0.10,0,0.00,0.00,0.02,0.00
2,10~19세,0,0,0,0,0,0,0,0,0,...,0,2.16,0.06,0.00,0.10,0,0.00,0.01,0.07,0.00
3,20~29세,0,0,0,0,0,0,0,0,0,...,0,7.66,0.42,0.00,0.35,0,0.00,0.14,0.32,0.10
4,30~39세,0,0,0,0,0,0,0,0,0,...,0,7.01,1.10,0.00,0.33,0,0.02,0.11,0.26,0.14
5,40~49세,0,0,0,0,0,0,0,0,0,...,0,4.12,1.41,0.00,0.33,0,0.07,0.07,0.67,0.07
6,50~59세,0,0,0,0,0,0,0,0,0,...,0,3.59,2.46,0.01,0.46,0,0.09,0.15,3.76,0.10
7,60~69세,0,0,0,0,0,0,0,0,0,...,0,1.67,3.50,0.00,0.29,0,0.27,0.05,7.87,0.02
8,70세 이상,0,0,0,0,0,0,0,0,0,...,0,0.26,2.19,0.00,0.37,0,0.03,0.03,15.58,0.00


# 설명 : make_age_dict(감염병)

2020년 7월(저번달, 특정시점)에 대해 지역별 감염병에 대한 데이터를 불러들여서  
1급,2급,3급으로 나눠진 기준을 통해 위험도 계수를 3,2,1 을 곱해주고  
10만명당 발생률 * 위험도 계수로, top5를 구함  
** 발생하지 않은 감염병은 제외됨 **


## tfidf 넣어서 연령별 요인 구하기

In [380]:
# tfidf 수정
def make_age_dict2(disease = '수두'):

    # 불러들이기
    age = pd.read_csv('연령_전체.csv', encoding = 'euc-kr',header = None)

    # 컬럼명 지정
    colum = ['나이','에볼라바이러스병 ','마버그열','라싸열','크리미안콩고출혈열','남아메리카출혈열','리프트밸리열','두창','페스트','탄저','보툴리눔독소증','야토병','신종감염병증후군',
            '중증급성호흡기증후군(SARS)','중동호흡기증후군(MERS)','동물인플루엔자 인체감염증','신종인플루엔자','디프테리아','수두','홍역','콜레라','장티푸스', '파라티푸스',
            '세균성이질','장출혈성대장균감염증','A형간염','백일해','유행성이하선염','풍진(2018년이전)','풍진(선천성)','풍진(후천성)','폴리오','수막구균 감염증','b형헤모필루스인플루엔자',
            '폐렴구균 감염증','한센병','성홍열','반코마이신내성황색포도알균(VRSA) 감염증','카바페넴내성장내세균속균종(CRE) 감염증','E형간염','파상풍','B형간염','일본뇌염',
            'C형간염','말라리아','레지오넬라증','비브리오패혈증','발진티푸스','발진열','쯔쯔가무시증','렙토스피라증','브루셀라증','공수병','신증후군출혈열','크로이츠펠트-야콥병(CJD) 및 변종크로이츠펠트-야콥병(vCJD)',
            '황열','뎅기열','큐열','웨스트나일열','라임병','진드기매개뇌염','유비저','치쿤구니야열','중증열성혈소판감소증후군(SFTS)','지카바이러스감염증']

    # age의 컬럼을 colum으로
    age.columns = colum


    ## 나이 요인 뽑기
#     age_dict = {}
#     for i in range(1, len(age['나이'])):
#         a =age['나이'][i]
#         #age_dict[a] =( age[disease][i] / np.log(1+age[disease][1:].sum()))
#         age_dict[a] = (age[disease][i]*100000) * (np.log(len(age['나이'])))/(1+((age[disease]*100000).sum()))

#     return age_dict

    age_dict = {}
    cnt = 0
    for i in range(1, len(age['나이'])):
        if age[disease][i] != 0:
            cnt += 1
    for i in range(1, len(age['나이'])):
        a =age['나이'][i]
        
        #age_dict[a] = np.log(1+age[disease][i]*100000) * (np.log(len(age['나이'])) / (1+cnt))
        #age_dict[a] = np.log(age[disease][i]*100000) * (np.log(len(age['나이'])) / (1+cnt))
        #age_dict[a] = (age[disease][i]*100000) * np.log(8/(1+cnt))
        #age_dict[a] = age[disease][i]* ((len(age['나이'])-1)/(1+cnt))
        age_dict[a] = age[disease][i]* ((len(age['나이'])-1)/(1+cnt))



    return age_dict

In [352]:
make_age_dict2('쯔쯔가무시증')

{'0~9세': 12.08,
 '10~19세': 10.862222222222222,
 '20~29세': 22.808888888888887,
 '30~39세': 31.368888888888886,
 '40~49세': 68.89777777777778,
 '50~59세': 223.20888888888888,
 '60~69세': 453.2444444444444,
 '70세 이상': 745.4222222222222}

# 설명 : make_risk_series(location)

2020년 7월(저번달, 특정시점)에 대해 지역별 감염병에 대한 데이터를 불러들여서  
1급,2급,3급으로 나눠진 기준을 통해 위험도 계수를 3,2,1 을 곱해주고  
10만명당 발생률 * 위험도 계수로, top5를 구함  
** 발생하지 않은 감염병은 제외됨 **


In [315]:
data2 = pd.read_csv('region_mean.csv')
data2_mean = data2.groupby(['구분'],as_index=False).mean()
data2_mean = data2_mean[['구분','에볼라바이러스병', '마버그열', '라싸열', '크리미안콩고출혈열',
       '남아메리카출혈열', '리프트밸리열', '두창', '페스트', '탄저', '보툴리눔독소증', '야토병', '신종감염병증후군',
       '중증급성호흡기증후군(SARS)', '중동호흡기증후군(MERS)', '동물인플루엔자 인체감염증', '신종인플루엔자',
       '디프테리아', '수두', '홍역', '콜레라', '장티푸스', '파라티푸스', '세균성이질', '장출혈성대장균감염증',
       'A형간염', '백일해', '유행성이하선염', '풍진(2018년이전)', '풍진(선천성)', '풍진(후천성)', '폴리오',
       '수막구균 감염증', 'b형헤모필루스인플루엔자', '폐렴구균 감염증', '한센병', '성홍열',
       '반코마이신내성황색포도알균(VRSA) 감염증', '카바페넴내성장내세균속균종(CRE) 감염증', 'E형간염', '파상풍',
       'B형간염', '일본뇌염', 'C형간염', '말라리아', '레지오넬라증', '비브리오패혈증', '발진티푸스', '발진열',
       '쯔쯔가무시증', '렙토스피라증', '브루셀라증', '공수병', '신증후군출혈열',
       '크로이츠펠트-야콥병(CJD) 및 변종크로이츠펠트-야콥병(vCJD)', '황열', '뎅기열', '큐열', '웨스트나일열',
       '라임병', '진드기매개뇌염', '유비저', '치쿤구니야열', '중증열성혈소판감소증후군(SFTS)', '지카바이러스감염증']]

In [322]:
data2_mean

,구분,에볼라바이러스병,마버그열,라싸열,크리미안콩고출혈열,남아메리카출혈열,리프트밸리열,두창,페스트,탄저,...,황열,뎅기열,큐열,웨스트나일열,라임병,진드기매개뇌염,유비저,치쿤구니야열,중증열성혈소판감소증후군(SFTS),지카바이러스감염증
0,강원강릉시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.029375,0.0
1,강원고성군,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.112812,0.0
2,강원동해시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.068750,0.0
3,강원삼척시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.138437,0.0
4,강원속초시,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.038125,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,충북청주시 상당구,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.105625,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
268,충북청주시 서원구,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.164062,0.0,0.0,0.0,0.000000,0.0,0.044687,0.0
269,충북청주시 청원구,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.015938,0.191250,0.0,0.0,0.0,0.000000,0.0,0.015938,0.0
270,충북청주시 흥덕구,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.036875,0.097500,0.0,0.0,0.0,0.000000,0.0,0.012188,0.0


In [317]:
data2_mean.to_csv('data2_mean.csv')

In [383]:
##### 10만명당 발생율 3년 평균값 사용하기
def make_risk_series2(location = '서울강남구'):
    # 위험도 가져오쟈
    급1 = ['에볼라바이러스병','마버그열','라싸열','크리미안콩고출혈열','남아메리카출혈열','리프트밸리열','두창','페스트','탄저','보툴리눔독소증','야토병','신종감염병증후군',
            '중증급성호흡기증후군(SARS)','중동호흡기증후군(MERS)','동물인플루엔자 인체감염증','신종인플루엔자','디프테리아']

    급2 = [ '수두','홍역','콜레라','장티푸스', '파라티푸스',
            '세균성이질','장출혈성대장균감염증','A형간염','백일해','유행성이하선염','풍진(2018년이전)','풍진(선천성)','풍진(후천성)','폴리오','수막구균 감염증','b형헤모필루스인플루엔자',
            '폐렴구균 감염증','한센병','성홍열','반코마이신내성황색포도알균(VRSA) 감염증','카바페넴내성장내세균속균종(CRE) 감염증','E형간염']


    급3 =  ['파상풍','B형간염','일본뇌염',
            'C형간염','말라리아','레지오넬라증','비브리오패혈증','발진티푸스','발진열','쯔쯔가무시증','렙토스피라증','브루셀라증','공수병','신증후군출혈열','크로이츠펠트-야콥병(CJD) 및 변종크로이츠펠트-야콥병(vCJD)',
            '황열','뎅기열','큐열','웨스트나일열','라임병','진드기매개뇌염','유비저','치쿤구니야열','중증열성혈소판감소증후군(SFTS)','지카바이러스감염증']




    ## input : 계절, 나이, 
    colum1 = 급1 + 급2 + 급3

    risk = {}
    for disease in colum1:
    #감염병 명 : 위험도
        k=0
        if disease in 급1:
            k = 3
        elif disease in 급2:
            k = 2
        else:
            k=1

        # 감염병 평균 가져오기
        data = pd.read_csv('data2_mean.csv')
        #data[disease] =  data[disease].apply(lambda x : 0 if x == '-' else float(x))
        # last = data[data['날짜'] == 202007]
        num = data[data['구분'] == '{}'.format(location)][disease]
        if num.values[0] != 0:
            risk[disease] = (num * k).values[0]


    risk_series = pd.Series(risk)
    
    return risk_series

In [333]:
make_risk_series2('서울노원구')

수두                                      18.924375
홍역                                       0.046875
장티푸스                                     0.045625
파라티푸스                                    0.023750
세균성이질                                    0.105625
장출혈성대장균감염증                               0.162500
A형간염                                     2.280000
백일해                                      0.161250
유행성이하선염                                  3.893125
수막구균 감염증                                 0.011250
폐렴구균 감염증                                 0.264375
성홍열                                      2.007500
B형간염                                     0.075000
일본뇌염                                     0.005625
C형간염                                     0.931250
말라리아                                     0.104063
레지오넬라증                                   0.115625
발진열                                      0.005938
쯔쯔가무시증                                   0.103125
렙토스피라증                                   0.005625


In [341]:
make_risk_series('전남화순군')

수두        3.16
A형간염      3.16
C형간염      1.58
쯔쯔가무시증    1.58
dtype: float64

In [384]:
make_risk_series2('전남화순군')

수두          20.026250
세균성이질        0.196250
A형간염         0.888750
백일해          1.380625
유행성이하선염      1.563125
폐렴구균 감염증     0.097500
성홍열          1.763125
B형간염         0.049375
일본뇌염         0.048750
C형간염         1.471562
레지오넬라증       0.048750
쯔쯔가무시증       3.036563
렙토스피라증       0.048750
신증후군출혈열      0.542812
큐열           0.098125
dtype: float64

In [307]:
data = pd.read_csv('지역별_2018부터_10만명당감염병발생비율.csv')
data

,Unnamed: 0,에볼라바이러스병,마버그열,라싸열,크리미안콩고출혈열,남아메리카출혈열,리프트밸리열,두창,페스트,탄저,...,웨스트나일열,라임병,진드기매개뇌염,유비저,치쿤구니야열,중증열성혈소판감소증후군(SFTS),지카바이러스감염증,날짜,구분,년
0,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,201801,서울강남구,2018
1,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,201801,서울강동구,2018
2,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,201801,서울강북구,2018
3,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,201801,서울강서구,2018
4,6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,201801,서울관악구,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699,9274,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,1.10,0.00,202008,제주서귀포시,2020
8700,9275,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,202008,제주제주시,2020
8701,9276,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,202008,제주남제주군,2020
8702,9277,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,202008,제주북제주군,2020


In [40]:
def make_risk_series(location = '서울강남구'):
    # 위험도 가져오쟈
    급1 = ['에볼라바이러스병 ','마버그열','라싸열','크리미안콩고출혈열','남아메리카출혈열','리프트밸리열','두창','페스트','탄저','보툴리눔독소증','야토병','신종감염병증후군',
            '중증급성호흡기증후군(SARS)','중동호흡기증후군(MERS)','동물인플루엔자 인체감염증','신종인플루엔자','디프테리아']

    급2 = [ '수두','홍역','콜레라','장티푸스', '파라티푸스',
            '세균성이질','장출혈성대장균감염증','A형간염','백일해','유행성이하선염','풍진(2018년이전)','풍진(선천성)','풍진(후천성)','폴리오','수막구균 감염증','b형헤모필루스인플루엔자',
            '폐렴구균 감염증','한센병','성홍열','반코마이신내성황색포도알균(VRSA) 감염증','카바페넴내성장내세균속균종(CRE) 감염증','E형간염']


    급3 =  ['파상풍','B형간염','일본뇌염',
            'C형간염','말라리아','레지오넬라증','비브리오패혈증','발진티푸스','발진열','쯔쯔가무시증','렙토스피라증','브루셀라증','공수병','신증후군출혈열','크로이츠펠트-야콥병(CJD) 및 변종크로이츠펠트-야콥병(vCJD)',
            '황열','뎅기열','큐열','웨스트나일열','라임병','진드기매개뇌염','유비저','치쿤구니야열','중증열성혈소판감소증후군(SFTS)','지카바이러스감염증']




    ## input : 계절, 나이, 
    colum1 = 급1 + 급2 + 급3

    risk = {}
    for disease in colum1:
    #감염병 명 : 위험도
        k=0
        if disease in 급1:
            k = 3
        elif disease in 급2:
            k = 2
        else:
            k=1

        # 2020년 7월 감염병 10만명당 발생률 가져오기 #    
        data = pd.read_csv('지역별_2018부터_10만명당감염병발생비율.csv')
        data[disease] =  data[disease].apply(lambda x : 0 if x == '-' else float(x))
        last = data[data['날짜'] == 202007]
        # last = data[data['날짜'] == data['날짜'].values.max()]
        num = last[last['구분'] == '{} '.format(location)][disease]
        if num.values[0] != 0:
            risk[disease] = (num * k).values[0]


    risk_series = pd.Series(risk)
    
    return risk_series

In [41]:
make_risk_series('서울노원구')

수두            5.58
장출혈성대장균감염증    0.74
A형간염          2.22
유행성이하선염       2.98
성홍열           0.38
dtype: float64

# 설명 : final_output(나이, 계절, 위치)
  
나이와 계절과 위치를 입력하면
특정 지역에 대한 risk_series의 위험도 * 계절요인 * 나이요인을 곱해서 나온  
가장 높은 위험도 3개를 예방수칙과 함께 보여줌  
  
  결론은 이것만 돌리면 위 함수들을 돌아서 결과가 나옴

In [319]:
def final_output(input_age = 41,input_season = '겨울', location = '서울강남구'):
    
    prevention = pd.read_excel('감염병 예방.xlsx')
    
    with open('season.pickle', 'rb') as f:
        season = pickle.load(f)
    
    if input_age < 10:
        input_age = '0~9세'
    elif input_age < 20:
        input_age = '10~19세'
    elif input_age < 30:
        input_age = '20~29세'
    elif input_age < 40:
        input_age = '30~39세'
    elif input_age < 50:
        input_age = '40~49세'
    elif input_age < 60:
        input_age = '50~59세'
    elif input_age < 70:
        input_age = '60~69세'
    else:
        input_age = '70세 이상'

    risk_series = make_risk_series(location)


    danger = {}
    for disease_names in list(risk_series.index):
        # 계절별 요인
        season_dict = make_season_dict(disease_names)##########위에거 사용
        num_season = season_dict[input_season]

        # 나이별 요인
        age_dict = make_age_dict(disease_names)############위에거 사용
        num_age = age_dict[input_age]

        danger[disease_names] = num_age* num_season*risk_series[disease_names]
        
    print('---------------------top3----------------------')
    print('{}, {}, {} 조심해야할 감염병 3개~'.format(input_age, input_season, location))
    print('')
    print(pd.Series(danger).sort_values()[::-1][:3])
    print('---------------------top3----------------------')
    print('')
    
    top3 = pd.Series(danger).sort_values()[::-1][:3]
    
    ## top3 예방수칙 보여주기 ##
    
    for disease_name in top3.index:
        for i in range(len(prevention)):
            if disease_name == prevention['질병'][i]: 
                print(' ※ {}의 예방 수칙 ※'.format(prevention['질병'][i]))
                print('')
                print(prevention['예방수칙'][i])
                print('')
                if prevention['백신'][i] =='O':
                    print('          ★백신을 접종하세요★')
                    print('')

    
    return pd.Series(danger).sort_values()[::-1]


In [386]:
## 지역별 평균값 가져온 거 사용하기

def final_output2(input_age = 41,input_season = '겨울', location = '서울강남구'):
    
    prevention = pd.read_excel('감염병 예방.xlsx')
    
    with open('season.pickle', 'rb') as f:
        season = pickle.load(f)
    
    if input_age < 10:
        input_age = '0~9세'
    elif input_age < 20:
        input_age = '10~19세'
    elif input_age < 30:
        input_age = '20~29세'
    elif input_age < 40:
        input_age = '30~39세'
    elif input_age < 50:
        input_age = '40~49세'
    elif input_age < 60:
        input_age = '50~59세'
    elif input_age < 70:
        input_age = '60~69세'
    else:
        input_age = '70세 이상'

    risk_series = make_risk_series2(location)


    danger = {}
    for disease_names in list(risk_series.index):
        # 계절별 요인
        season_dict = make_season_dict2(disease_names)##########위에거 사용
        num_season = season_dict[input_season]

        # 나이별 요인
        age_dict = make_age_dict2(disease_names)############위에거 사용
        num_age = age_dict[input_age]

        danger[disease_names] = num_age* num_season*risk_series[disease_names]
        
    print('---------------------top3----------------------')
    print('{}, {}, {} 조심해야할 감염병 3개~'.format(input_age, input_season, location))
    print('')
    print(pd.Series(danger).sort_values()[::-1][:3])
    print('---------------------top3----------------------')
    print('')
    
    top3 = pd.Series(danger).sort_values()[::-1][:3]
    
    ## top3 예방수칙 보여주기 ##
    
    for disease_name in top3.index:
        for i in range(len(prevention)):
            if disease_name == prevention['질병'][i]: 
                print(' ※ {}의 예방 수칙 ※'.format(prevention['질병'][i]))
                print('')
                print(prevention['예방수칙'][i])
                print('')
                if prevention['백신'][i] =='O':
                    print('          ★백신을 접종하세요★')
                    print('')

    
    return pd.Series(danger).sort_values()[::-1]


In [338]:
# 수두
a = final_output2(input_age = 15, input_season = '여름', location = '서울중구')

---------------------top3----------------------
10~19세, 여름, 서울중구 조심해야할 감염병 3개~

수두         9.623506e+08
유행성이하선염    8.015248e+07
성홍열        1.455756e+06
dtype: float64
---------------------top3----------------------

 ※ 수두의 예방 수칙 ※

외출 후엔 반드시 손발을 씻고 양치질을 합니다.
수두환자와 접촉하지 않도록 주의합니다.
수두 생백신으로 예방 접종 가능
수두에 노출되고 3일 이내에 백신을 접종 받으면 발병을 예방하거나 증상을 완화

          ★백신을 접종하세요★

 ※ 유행성이하선염의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
예방접종을 반드시 맞는다.

          ★백신을 접종하세요★

 ※ 성홍열의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
발열 및 호흡기 증상이 있으면 마스크 쓰기



In [369]:
# 수두
a = final_output2(input_age = 75, input_season = '여름', location = '서울중구')

---------------------top3----------------------
70세 이상, 여름, 서울중구 조심해야할 감염병 3개~

C형간염        4.300127
폐렴구균 감염증    1.072344
A형간염        0.462396
dtype: float64
---------------------top3----------------------

 ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.

 ※ 폐렴구균 감염증의 예방 수칙 ※

예방접종을 반드시 맞는다.
19세 미만 소아청소년 : 단백결합 백식
19세 이상 성인 : 다당 백신 접종

          ★백신을 접종하세요★

 ※ A형간염의 예방 수칙 ※

개인위생 관리.
 끓인 물을 마시거나 충분한 익힌 음식을 섭취.
 외출 후에는 반드시 손을 깨끗이 씻는다.
예방접종을 반드시 맞는다.

          ★백신을 접종하세요★



In [364]:
b = final_output(input_age = 75, input_season = '여름', location = '서울노원구')

---------------------top3----------------------
70세 이상, 여름, 서울노원구 조심해야할 감염병 3개~

C형간염        1.965697
폐렴구균 감염증    0.474976
A형간염        0.408234
dtype: float64
---------------------top3----------------------

 ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.

 ※ 폐렴구균 감염증의 예방 수칙 ※

예방접종을 반드시 맞는다.
19세 미만 소아청소년 : 단백결합 백식
19세 이상 성인 : 다당 백신 접종

          ★백신을 접종하세요★

 ※ A형간염의 예방 수칙 ※

개인위생 관리.
 끓인 물을 마시거나 충분한 익힌 음식을 섭취.
 외출 후에는 반드시 손을 깨끗이 씻는다.
예방접종을 반드시 맞는다.

          ★백신을 접종하세요★



In [379]:
make_season_dict('쯔쯔가무시증')

{'가을': 1.2266134729082279,
 '겨울': 0.08435184209068997,
 '봄': 0.03125816407235379,
 '여름': 0.04405349708528614}

In [378]:
make_risk_series2('전남화순군')

수두          20.026250
세균성이질        0.196250
A형간염         0.888750
백일해          1.380625
유행성이하선염      1.563125
폐렴구균 감염증     0.097500
성홍열          1.763125
B형간염         0.049375
일본뇌염         0.048750
C형간염         1.471562
레지오넬라증       0.048750
쯔쯔가무시증       3.036563
렙토스피라증       0.048750
신증후군출혈열      0.542812
큐열           0.098125
dtype: float64

In [387]:
ㅊ = final_output2(input_age = 71, input_season = '가을', location = '전남화순군')

---------------------top3----------------------
70세 이상, 가을, 전남화순군 조심해야할 감염병 3개~

쯔쯔가무시증    1.796681e+08
수두        4.971329e+07
C형간염      3.130363e+06
dtype: float64
---------------------top3----------------------

 ※ 쯔쯔가무시증의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
2차감염 예방을 위하여 긁지 않도록 하고 손톱을 짧게 깍아준다.
예방접종을 반드시 맞는다.

          ★백신을 접종하세요★

 ※ 수두의 예방 수칙 ※

외출 후엔 반드시 손발을 씻고 양치질을 합니다.
수두환자와 접촉하지 않도록 주의합니다.
수두 생백신으로 예방 접종 가능
수두에 노출되고 3일 이내에 백신을 접종 받으면 발병을 예방하거나 증상을 완화

          ★백신을 접종하세요★

 ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.



In [377]:
ㅊ = final_output2(input_age = 71, input_season = '가을', location = '전남화순군')

---------------------top3----------------------
70세 이상, 가을, 전남화순군 조심해야할 감염병 3개~

쯔쯔가무시증     3.567406
C형간염       0.448256
신증후군출혈열    0.389294
dtype: float64
---------------------top3----------------------

 ※ 쯔쯔가무시증의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
2차감염 예방을 위하여 긁지 않도록 하고 손톱을 짧게 깍아준다.
예방접종을 반드시 맞는다.

          ★백신을 접종하세요★

 ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.

 ※ 신증후군출혈열의 예방 수칙 ※

유행 지역의 산이나 풀밭에 가는 것을 피할 것.
늦가을(10~11월)과 늦봄(5~6월) 건조기에는 절대 잔디 위에 눕거나 잠을 자지 말 것
들쥐의 배설물에 접촉을 피할 것
잔디 위에 침구나 옷을 말리지 말 것
야외활동 후 귀가시에는 옷에 묻은 먼지를 털고 목욕을 할 것
가능한 한 피부의 노출을 적게 할 것
전염위험이 높은 사람(군인, 농부 등)은 적기에 예방접종을 받을 것

          ★백신을 접종하세요★



In [280]:
ㅊ = final_output(input_age = 71, input_season = '가을', location = '전남화순군')

---------------------top3----------------------
70세 이상, 가을, 전남화순군 조심해야할 감염병 3개~

쯔쯔가무시증    6.647883e+12
수두        5.578243e+11
C형간염      2.390070e+11
dtype: float64
---------------------top3----------------------

  ※ 쯔쯔가무시증의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
2차감염 예방을 위하여 긁지 않도록 하고 손톱을 짧게 깍아준다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★

  ※ 수두의 예방 수칙 ※

외출 후엔 반드시 손발을 씻고 양치질을 합니다.
수두환자와 접촉하지 않도록 주의합니다.
수두 생백신으로 예방 접종 가능
수두에 노출되고 3일 이내에 백신을 접종 받으면 발병을 예방하거나 증상을 완화

     ★백신을 접종하세요★

  ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.



In [228]:
ㅊ = final_output(input_age = 71, input_season = '가을', location = '전남함평군')

---------------------top3----------------------
70세 이상, 가을, 전남함평군 조심해야할 감염병 3개~

C형간염          2.737991e+07
장출혈성대장균감염증    2.830135e+03
큐열            7.195814e+02
dtype: float64
---------------------top3----------------------

  ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.

  ※ 장출혈성대장균감염증의 예방 수칙 ※

도축장에서는 고기가 동물의 부산물에 의해 오염되지 않게 주의한다.
우유는 살균하도록 하며, 고기는 충분히 익혀서 먹는다.
간이상수도 및 수영장의 염소 소독을 철저히 한다.
모유를 먹이는 것이 좋으며, 우유를 탈 때 쓰는 물은 반드시 끓여 쓴다.
유아원, 양로원 등 보호시설과 보육시설의 종사자 및 수용자에 대한 개 인 위생 수칙을 준수하도록 교육한다.
과일, 야채는 깨끗한 물에 충분히 씻어서 먹는다.



In [122]:
ㅊ = final_output(input_age = 71, input_season = '가을', location = '전남화순군')

---------------------top3----------------------
70세 이상, 가을, 전남화순군 조심해야할 감염병 3개~

쯔쯔가무시증    4.684161
C형간염      3.270774
A형간염      0.531995
dtype: float64
---------------------top3----------------------

  ※ 쯔쯔가무시증의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
2차감염 예방을 위하여 긁지 않도록 하고 손톱을 짧게 깍아준다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★

  ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.

  ※ A형간염의 예방 수칙 ※

개인위생 관리.
 끓인 물을 마시거나 충분한 익힌 음식을 섭취.
 외출 후에는 반드시 손을 깨끗이 씻는다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★



In [208]:
ㅊ = final_output(input_age = 41, input_season = '가을', location = '대구중구')

---------------------top3----------------------
40~49세, 가을, 대구중구 조심해야할 감염병 3개~

수두         1.993864e+12
유행성이하선염    3.314770e+11
C형간염       1.152590e+11
dtype: float64
---------------------top3----------------------

  ※ 수두의 예방 수칙 ※

외출 후엔 반드시 손발을 씻고 양치질을 합니다.
수두환자와 접촉하지 않도록 주의합니다.
수두 생백신으로 예방 접종 가능
수두에 노출되고 3일 이내에 백신을 접종 받으면 발병을 예방하거나 증상을 완화

     ★백신을 접종하세요★

  ※ 유행성이하선염의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★

  ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.



In [16]:
k = final_output(input_age = 51, input_season = '가을', location = '서울노원구')

---------------------top3----------------------
50~59세, 가을, 서울노원구 조심해야할 감염병 3개~

A형간염          0.033705
유행성이하선염       0.009190
장출혈성대장균감염증    0.007801
dtype: float64
---------------------top3----------------------

※ A형간염의 예방 수칙 ※

개인위생 관리.
 끓인 물을 마시거나 충분한 익힌 음식을 섭취.
 외출 후에는 반드시 손을 깨끗이 씻는다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★

※ 유행성이하선염의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★

※ 장출혈성대장균감염증의 예방 수칙 ※

도축장에서는 고기가 동물의 부산물에 의해 오염되지 않게 주의한다.
우유는 살균하도록 하며, 고기는 충분히 익혀서 먹는다.
간이상수도 및 수영장의 염소 소독을 철저히 한다.
모유를 먹이는 것이 좋으며, 우유를 탈 때 쓰는 물은 반드시 끓여 쓴다.
유아원, 양로원 등 보호시설과 보육시설의 종사자 및 수용자에 대한 개 인 위생 수칙을 준수하도록 교육한다.
과일, 야채는 깨끗한 물에 충분히 씻어서 먹는다.



In [40]:
df.groupby('년').sum().reset_index()

,년,에볼라바이러스병,마버그열,라싸열,크리미안콩고출혈열,남아메리카출혈열,리프트밸리열,두창,페스트,탄저,...,황열,뎅기열,큐열,웨스트나일열,라임병,진드기매개뇌염,유비저,치쿤구니야열,중증열성혈소판감소증후군(SFTS),지카바이러스감염증
0,2010,0,0,0,0,0,0,0,0,0,...,0,125,13,0,0,0,0,0,0,0
1,2011,0,0,0,0,0,0,0,0,0,...,0,72,8,0,2,0,1,0,0,0
2,2012,0,0,0,0,0,0,0,0,0,...,0,149,10,1,3,0,0,0,0,0
3,2013,0,0,0,0,0,0,0,0,0,...,0,252,11,0,11,0,2,2,36,0
4,2014,0,0,0,0,0,0,0,0,0,...,0,165,8,0,13,0,2,1,55,0
5,2015,0,0,0,0,0,0,0,0,0,...,0,255,27,0,9,0,4,2,79,0
6,2016,0,0,0,0,0,0,0,0,0,...,0,313,81,0,27,0,4,10,165,16
7,2017,0,0,0,0,0,0,0,0,0,...,0,171,96,0,31,0,2,5,272,11
8,2018,0,0,0,0,0,0,0,0,0,...,0,159,163,0,23,0,2,3,259,3
9,2019,0,0,0,0,0,0,0,0,0,...,0,273,162,0,23,0,8,16,223,3


In [70]:
d = final_output(input_age = 15, input_season = '가을', location = '충북청주시 청원구')

---------------------top3----------------------
60~69세, 가을, 충북청주시 청원구 조심해야할 감염병 3개~

C형간염       0.034961
유행성이하선염    0.010115
A형간염       0.006118
dtype: float64
---------------------top3----------------------

  ※ C형간염의 예방 수칙 ※

환자의 개인 세면도구(칫솔, 면도기, 손톱 깎기)는 따로 사용
주사바늘이나 피어싱 기구, 침 등을 공동으로 사용금지
감염자와 성교시 가급적이면 콘돔을 사용하도록 한다.

  ※ 유행성이하선염의 예방 수칙 ※

사람이 많이 모인곳이나 밀폐된 곳을 피한다.
외출 후 곧바로 손발을 씻고 양치질을 한다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★

  ※ A형간염의 예방 수칙 ※

개인위생 관리.
 끓인 물을 마시거나 충분한 익힌 음식을 섭취.
 외출 후에는 반드시 손을 깨끗이 씻는다.
예방접종을 반드시 맞는다.

     ★백신을 접종하세요★



In [148]:
df[(df['날짜'] ==  '20-01') & (df['구분'] == '전남화순군 ')]['수두']

29592    18
Name: 수두, dtype: int64

In [232]:
data = pd.read_csv('지역별_2018부터_10만명당감염병발생비율.csv')
data['쯔쯔가무시증'] =  data['쯔쯔가무시증'].apply(lambda x : 0 if x == '-' else float(x))
last = data[data['날짜'] == 202007]


In [249]:
last[last['구분'] =='전남화순군 ']['수두']

8376    1.58
Name: 수두, dtype: object

In [303]:

season_dict = {}
for i in range(len(season['계절'])):
    a =season['계절'][i]
    #season_dict[a] =( season[disease][i] / np.log(1+season[disease].sum())
    season_dict[a] =(season['수두'][i] * np.log(len(season['계절'])))/(1+season['수두'].sum())


In [304]:
season_dict

{'가을': 0.31395569175680854,
 '겨울': 0.43201437243734214,
 '봄': 0.3395112284086948,
 '여름': 0.3008104530696342}

In [305]:

season_dict = {}
for i in range(len(season['쯔쯔가무시증'])):
    a =season['계절'][i]
    #season_dict[a] =( season[disease][i] / np.log(1+season[disease].sum())
    season_dict[a] =(season['쯔쯔가무시증'][i] * np.log(len(season['계절'])))/(1+season['쯔쯔가무시증'].sum())


In [306]:
season_dict

{'가을': 1.2266134729082279,
 '겨울': 0.08435184209068997,
 '봄': 0.03125816407235379,
 '여름': 0.04405349708528614}

In [367]:
make_age_dict('폐렴구균 감염증')

{'0~9세': 0.15734179907055565,
 '10~19세': 0.015701468514317408,
 '20~29세': 0.009486303894066767,
 '30~39세': 0.03696387379412223,
 '40~49세': 0.0673854690406122,
 '50~59세': 0.19267010322776984,
 '60~69세': 0.46450177688189,
 '70세 이상': 1.0775132767950322}